# **Homework 1: COVID-19 Cases Prediction (Regression)**

# Import packages

In [1]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# K Fold
import sklearn
from sklearn.model_selection import KFold

# Some Utility Functions

You do not need to modify this part.

In [2]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds

# Dataset

In [3]:
import torch
from torch.utils.data import Dataset
import numpy as np

class PM_25(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        # Ensure x is of a uniform data type compatible with PyTorch
        if x.dtype == object:
            x = x.astype(float)  # convert to float if possible
        self.x = torch.FloatTensor(x)

        if y is not None:
            if y.dtype == object:
                y = y.astype(float)  # convert y to float if possible
            self.y = torch.FloatTensor(y)
        else:
            self.y = y

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)


# Neural Network Model
Try out different model architectures by modifying the class below.

In [4]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [5]:
import numpy as np
def select_feat(train_set, valid_set, test_data, select_all):
    # Assuming that 'No' is the first column and 'pm2.5' is the sixth column in train_set and valid_set
    # For test_data, 'No' is the first column, but there is no 'pm2.5' column

    # Indices to keep for features (excluding 'No' and 'pm2.5')
    feature_indices = list(range(1, 5)) + list(range(6, train_set.shape[1]))  # Excluding first and sixth columns

    if select_all:
        # Select all features except 'No' and 'pm2.5' for train and valid sets
        x_train = train_set[:, feature_indices]
        x_valid = valid_set[:, feature_indices]

        # For test_data, exclude only 'No' (the first column)
        x_test = test_data[:, feature_indices]
    else:
        # Implement other selection criteria if needed
        pass

    # Extracting the target variable 'pm2.5' (index 5) for train and valid sets
    y_train = train_set[:, 5]
    y_valid = valid_set[:, 5]

    return x_train, x_valid, x_test, y_train, y_valid



# Training Loop

In [6]:
def trainer(train_load, valid_load, model, config, device):
    min_loss=1e9
    Loss_min=[]
    Loss_mean=[]
    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.
    #m=nn.BatchNorm1d(num_features=11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=device)
    train_loader=train_load
    valid_loader=valid_load
    # Define your optimization algorithm.
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.Adam(model.parameters())

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device.
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)
                if loss<min_loss:
                  min_loss=loss

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return mean_valid_loss
    Loss_min.append(min_loss)
    Loss_mean.append(mean_valid_loss)
    return mean_valid_loss

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 6000,     # Number of epochs.
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [8]:
import numpy as np
import torch
import pandas as pd

# Load the data as Pandas DataFrames
train_df = pd.read_csv('Data/train.csv')
test_df = pd.read_csv('Data/test.csv')

# Perform one-hot encoding on the DataFrames
train_df = pd.get_dummies(train_df, columns=['cbwd'])
test_df = pd.get_dummies(test_df, columns=['cbwd'])

# Now, you can convert them to NumPy arrays if needed
train_data = train_df.values
test_data = test_df.values

# Start training!

In [9]:
kfold = sklearn.model_selection.KFold(n_splits=5, random_state=True, shuffle=True)
best_model = My_Model(input_dim=14)
best_loss = math.inf
for fold, (train_ids, valid_ids) in enumerate(kfold.split(train_data)):
    # Print out the data size.
    print(f"train_data size: {train_data.shape}")
    print(f"first 10 line of train_data:\n {train_data[:10]}")
    #valid_data size: {valid_data.shape}
    # test_data size: {test_data.shape})
    # Sample elements randomly from a given list of ids, no replacement.
    train_set,  valid_set= train_data[train_ids, :], train_data[valid_ids, :]
    # Select features
    x_train, x_valid, x_test, y_train, y_valid = select_feat(train_set, valid_set, test_data, config['select_all'])

    train_dataset, valid_dataset, test_dataset = PM_25(x_train, y_train), \
                                                PM_25(x_valid, y_valid), \
                                                PM_25(x_test)

    # Pytorch data loader loads pytorch dataset into batches.
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
    model = My_Model(input_dim=14).to(device) # put your model and data on the same computation device.
    loss = trainer(train_loader, valid_loader, model, config, device)
    if loss < best_loss:
      best_model = model
      best_loss = loss

print(f'best loss = {best_loss}')
"""
import statistics
print("Statistics area:\n")
for i in range(len(Loss_mean)):
  print("iteration {i} epoch= ,"+": mean loss="+Loss_mean[i]+", min loss="+Loss_min[i])

statistics.pstdev(Loss_mean)
"""

train_data size: (39504, 16)
first 10 line of train_data:
 [[1 2010 1 1 0 -1 -21 -11 1021.0 1.79 0 0 False True False False]
 [2 2010 1 1 1 -1 -21 -12 1020.0 4.92 0 0 False True False False]
 [3 2010 1 1 2 -1 -21 -11 1019.0 6.71 0 0 False True False False]
 [4 2010 1 1 3 -1 -21 -14 1019.0 9.84 0 0 False True False False]
 [5 2010 1 1 4 -1 -20 -12 1018.0 12.97 0 0 False True False False]
 [6 2010 1 1 5 -1 -19 -10 1017.0 16.1 0 0 False True False False]
 [7 2010 1 1 6 -1 -19 -9 1017.0 19.23 0 0 False True False False]
 [8 2010 1 1 7 -1 -19 -9 1017.0 21.02 0 0 False True False False]
 [9 2010 1 1 8 -1 -19 -9 1017.0 24.15 0 0 False True False False]
 [10 2010 1 1 9 -1 -20 -8 1017.0 27.28 0 0 False True False False]]


Epoch [1/6000]: 100%|██████████| 124/124 [00:05<00:00, 21.61it/s, loss=7.78e+3]


Epoch [1/6000]: Train loss: 8523.1834, Valid loss: 8738.5143
Saving model with loss 8738.514...


Epoch [2/6000]: 100%|██████████| 124/124 [00:00<00:00, 301.26it/s, loss=6.54e+3]


Epoch [2/6000]: Train loss: 8271.9088, Valid loss: 8440.4608
Saving model with loss 8440.461...


...

Epoch [2013/6000]: 100%|██████████| 124/124 [00:00<00:00, 160.65it/s, loss=2.27e+3]


Epoch [2013/6000]: Train loss: 4363.9822, Valid loss: 4027.9841


Epoch [2014/6000]: 100%|██████████| 124/124 [00:00<00:00, 154.79it/s, loss=3.9e+3] 


Epoch [2014/6000]: Train loss: 4373.1622, Valid loss: 4150.0009


Epoch [2015/6000]:  27%|██▋       | 34/124 [00:00<00:00, 148.55it/s, loss=4.07e+3]

# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6006 (pid 31880), started 1:18:17 ago. (Use '!kill 31880' to kill it.)

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=14).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

100%|██████████| 17/17 [00:00<00:00, 1545.47it/s]
